In [1]:
# Data reader

In [2]:
# import pandas as pd

In [3]:
# import ray
# ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})
# import modin.pandas as pd

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark.sql import SparkSession
import pyspark.pandas as pd
pd.set_option('compute.default_index_type', 'distributed')

spark = SparkSession.builder \
.master('local[*]') \
.config("spark.driver.memory", "10g") \
.getOrCreate()

23/03/01 12:38:38 WARN Utils: Your hostname, abinbev resolves to a loopback address: 127.0.1.1; using 192.168.0.105 instead (on interface wlp0s20f3)
23/03/01 12:38:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/01 12:38:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/01 12:38:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
select_cols = ['a', 'b']
data_type_parser = {'b': 'int8', 'a': 'int8'}                                  

def is_consistent_cols_and_parser(select_cols, data_type_parser):
    if select_cols is None:
        return True
    if data_type_parser is None:
        return False
    if set(select_cols) != set(data_type_parser.keys()):
        return False
    return True

def read_csv_with_parser(file_path, select_cols=None, data_type_parser=None, compression = None):
    if not is_consistent_cols_and_parser(select_cols, data_type_parser):
        raise ValueError('select_cols and data_type_parser are not consistent')
    if select_cols is None:
        return pd.read_csv(file_path, compression = compression)
    if data_type_parser is None:
        return pd.read_csv(file_path, usecols=select_cols, compression = compression)
    return pd.read_csv(file_path, usecols=select_cols, dtype=data_type_parser, compression = compression)
try:
    df = read_csv_with_parser('test.csv', select_cols, data_type_parser)
    df_gzip = read_csv_with_parser('test.csv.gzip', select_cols, data_type_parser, compression = 'gzip')
except Exception as e:
    pass

def get_df():
    return pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})

def get_df_time():
    return pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': ['2019-01-01', '2019-01-02', '2019-01-03'], 'd': ['2019-01-01', '2019-01-02', '2019-01-03']})

def get_df_time_str():
    return pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6], 'c': ['2019-01-01', '2019-01-02', '2019-01-03'], 'd': ['2019-01-01', '2019-01-02', '2019-01-03'], 'e': ['a', 'b', 'c']})

* select columns and keys of data type parser has to be same.
* values of data type parser has to be minimal data type of the column.
* TODO: Write something similar for parquet and parquet-gzip

In [6]:
# Data parser
df = get_df()
df_date = get_df_time()

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [7]:
select_cols = ['a', 'b', 'c', 'd']
data_type_parser = {'b': 'int8', 'a': 'int8', 'c': 'str', 'd': 'str'}
date_formats = {'d': '%Y-%m-%d', 'c': '%Y-%m-%d'}

In [8]:
def parse_date(df, date_formats):
    for col in date_formats.keys():
        df[col] = pd.to_datetime(df[col], format=date_formats[col])
    return df

try:
    df_date = read_csv_with_parser('test_date.csv')
except:
    df_date = get_df_time()
df_date = parse_date(df_date, date_formats)
df_date

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/py

,a,b,c,d
0,1,4,2019-01-01,2019-01-01
1,2,5,2019-01-02,2019-01-02
2,3,6,2019-01-03,2019-01-03


In [9]:
def filter_by_range_with_eq(df, rage_filter):
    for col in rage_filter.keys():
        df = df[(df[col] >= rage_filter[col][0]) & (df[col] <= rage_filter[col][1])]
    return df

rage_filter = {'a': [1, 2], 'b': [0, 4]}
df_date = filter_by_range_with_eq(df_date, rage_filter)
df_date

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instea

,a,b,c,d
0,1,4,2019-01-01,2019-01-01


In [10]:
try:
    df_date = read_csv_with_parser('test_date.csv', select_cols, data_type_parser)
except:
    df_date = get_df_time()
df_date = parse_date(df_date, date_formats)
range_filter = {'a': [1, 3], 'b': [0, 6], 'c': ['2019-01-01', '2019-01-02']}
df_date = filter_by_range_with_eq(df_date, range_filter)
df_date

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/py

,a,b,c,d
0,1,4,2019-01-01,2019-01-01
1,2,5,2019-01-02,2019-01-02


In [11]:
def filter_by_str(df, str_filter):
    for col in str_filter.keys():
        df = df[df[col].isin(str_filter[col])]
    return df

In [12]:
str_filter = {'e': ['a', 'b']}
try:
    df_date = read_csv_with_parser('test_date_str.csv')
except:
    df_date = get_df_time_str()
df_date = filter_by_str(df_date, str_filter)
df_date

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


,a,b,c,d,e
0,1,4,2019-01-01,2019-01-01,a
1,2,5,2019-01-02,2019-01-02,b


In [13]:
def get_unique_str_date(df):
    str_cols = df.select_dtypes(include=['object']).columns.tolist()
    date_cols = df.select_dtypes(include=['datetime64[ns]']).columns.tolist()
    cols_to_check = str_cols + date_cols
    unique_dict = {}
    for col in cols_to_check:
        unique_dict[col] = df[col].unique().tolist()
    return unique_dict

try:
    df_date = read_csv_with_parser('test_date_str.csv')
except:
    df_date = get_df_time_str()
get_unique_str_date(df_date)

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is exp

{'c': ['2019-01-01', '2019-01-02', '2019-01-03'],
 'd': ['2019-01-01', '2019-01-02', '2019-01-03'],
 'e': ['a', 'b', 'c']}

In [14]:
def get_min_max_num_date(df, date_formats='%Y-%m-%d'):
    min_max_dict = {}
    # TODO: This does not work in Pandas for Spark
    num_cols = df.select_dtypes('number').columns.tolist()
    date_cols = df.select_dtypes('datetime64[ns]').columns.tolist()
    cols_check = num_cols + date_cols
    for col in cols_check:
        if col in num_cols:
            min_max_dict[col] = {"min": df[col].min(), "max": df[col].max()}
        if col in date_cols:
            min_max_dict[col] = {"min": df[col].min().strftime(date_formats), "max": df[col].max().strftime(date_formats)}
    return min_max_dict

try:
    df_date = read_csv_with_parser('test_date.csv', select_cols, data_type_parser)
except:
    df_date = get_df_time()
df_date = parse_date(df_date, date_formats)
get_min_max_num_date(df_date)

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/py

{'c': {'min': '2019-01-01', 'max': '2019-01-03'},
 'd': {'min': '2019-01-01', 'max': '2019-01-03'}}

In [15]:
def get_unique_count_str_date(df):
    unique_dict = {}
    str_cols = df.select_dtypes('object').columns.tolist()
    date_cols = df.select_dtypes('datetime64[ns]').columns.tolist()
    cols_check = str_cols + date_cols
    for col in cols_check:
        unique_dict[col] = df[col].nunique()
    return unique_dict
try:
    df_date = read_csv_with_parser('test_date_str.csv')
except:
    df_date = get_df_time_str()
get_unique_count_str_date(df_date)

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


{'c': 3, 'd': 3, 'e': 3}

In [16]:
def infer_freq(df, date_cols):
    freq = {}
    for col in date_cols:
        try:
            freq[col] = pd.infer_freq(df[col])
        except Exception as e:
            freq[col] = None
    return freq
try:
    df_date = read_csv_with_parser('test_date.csv', select_cols, data_type_parser)
except:
    df_date = get_df_time()
df_date = parse_date(df_date, date_formats)
infer_freq(df_date, ['c'])

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


{'c': None}

In [17]:
def check_duplicate_str_date(df):
    str_cols = df.select_dtypes('object').columns.tolist()
    date_cols = df.select_dtypes('datetime64[ns]').columns.tolist()
    cols_check = str_cols + date_cols
    duplicate_status = {}
    for col in cols_check:
        duplicate_status[col] = df[[col]].duplicated().any()
    return duplicate_status
try:
    df_date = read_csv_with_parser('test_date_str.csv')
except:
    df_date = get_df_time_str()
check_duplicate_str_date(df_date)

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


{'c': False, 'd': False, 'e': False}

In [18]:
def check_missing_cols(df, cols):
    missing_cols = []
    for col in cols:
        if col not in df.columns:
            missing_cols.append(col)
    return missing_cols

def log_missing_cols(df, cols):
    missing_cols = check_missing_cols(df, cols)
    if len(missing_cols) > 0:
        print(f"Missing columns: {missing_cols}")
    else:
        print("No missing columns")

try:
    df_date = read_csv_with_parser('test_date_str.csv')
except:
    df_date = get_df_time_str()
log_missing_cols(df_date, ['a', 'b', 'c', 'd', 'e'])
log_missing_cols(df_date, ['a', 'b', 'c', 'd', 'e', 'f'])

No missing columns
Missing columns: ['f']


/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [19]:
def basic_schema_df(df):
    # TODO: This does not work in Pandas for Spark
    num_cols = df.select_dtypes('number').columns.tolist()
    str_cols = df.select_dtypes('object').columns.tolist()
    date_cols = df.select_dtypes('datetime64[ns]').columns.tolist()
    return {"num_cols": num_cols, "str_cols": str_cols, "date_cols": date_cols}
try:
    df_date = read_csv_with_parser('test_date_str.csv')
except:
    df_date = get_df_time_str()
df_date = parse_date(df_date, date_formats)
basic_schema = basic_schema_df(df_date)
basic_schema

/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/home/abinbev/anaconda3/envs/aikit-modin/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


{'num_cols': [], 'str_cols': ['e'], 'date_cols': ['c', 'd']}